# GRiTS

In [1]:
## QtCore, needed for interactive visualization, will open a separate python process
#from PySide2 import QtCore
#
#%gui qt

In [2]:
import warnings

#import fresnel
#import fresnel.interact
import freud
import matplotlib.cm
import mbuild as mb
import numpy as np
import PIL

import utils
from cg_compound import CG_Compound

/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [3]:
p3ht = mb.load("molfiles/P3HT_16.mol2")
p3ht.translate_to(np.zeros(3))

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    p3ht_mol = p3ht.to_pybel(box = p3ht.boundingbox)
    p3ht_mol.OBMol.PerceiveBondOrders()

In [4]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    cg_beads = [("_B", "c1sccc1"), ("_S", "CCC")]
    cg_p3ht = utils.coarse(p3ht_mol, cg_beads)
    p3ht_colors = {
        "_B" : 'blue',
        "_S" : 'orange'
    }
    
    cg_p3ht.visualize(color_scheme=p3ht_colors, show_atomistic=True).show()
    cg_p3ht.visualize(color_scheme=p3ht_colors).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [77]:
def get_bonded(compound, particle):
    def is_particle(i,j):
        if i is particle:
            return j
        elif j is particle:
            return i
        else:
            return False
    return [is_particle(i,j) for i,j in compound.bonds() if is_particle(i,j)]


def get_index(compound, particle):
    return [p for p in compound.particles()].index(particle)

def remove_hydrogen(compound, particle):
    hydrogens = [i for i in get_bonded(compound, particle) if i.name == "H"]
    if hydrogens:
        compound.remove(hydrogens[0])

def backmap(cg_compound, bead_dict, bond_dict):
    fine_grained = mb.Compound()
    #bead_lengths = [0]
    anchors = dict()
    for i,bead in enumerate(cg_p3ht.particles()):
        smiles = bead_dict[bead.name]["smiles"]
        b = mb.load(smiles, smiles=True)
        b.translate_to(bead.pos)
        anchors[i] = dict()
        for index in bead_dict[bead.name]["anchors"]:
            anchors[i][index] = b[index]
            
        fine_grained.add(b)
        #bead_lengths.append(fine_grained.n_particles)
    atoms = []
    for ibead,jbead in cg_compound.bonds():
        i = get_index(cg_compound, ibead)
        j = get_index(cg_compound, jbead)
        names = [ibead.name,jbead.name]
        bondname = "".join(names)
        try:
            fi,fj = bond_dict[bondname]
        except KeyError:
            try:
                bondname = "".join(names[::-1])
                fj,fi = bond_dict[bondname]
            except KeyError:
                print(f"{bondname} not defined in bond dictionary.")
                raise
        iatom = anchors[i][fi]
        jatom = anchors[j][fj]
        if (iatom in atoms) or (jatom in atoms):
            try:
                iatom = anchors[i][fj]
                jatom = anchors[j][fi]
            except KeyError:
                pass
            if (iatom in atoms) or (jatom in atoms):
                print("hmm something went wrong")
        fine_grained.add_bond((iatom, jatom))

        atoms.append(iatom)
        atoms.append(jatom)
    
    for atom in atoms:
        remove_hydrogen(fine_grained,atom)

    return fine_grained

In [78]:
bead_dict = {
    "_B": {
        "smiles": "c1sccc1", 
        "anchors": [0,2,4], 
    },
    "_S": {
        "smiles": "CCC",
        "anchors": [0,2],
    }
}

bond_dict = {
    "_B_B": (0,2), 
    "_B_S": (4,0),
    "_S_S": (2,0)
}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    fine_grained = backmap(cg_p3ht, bead_dict, bond_dict)
    fine_grained.visualize().show()

hmm something went wrong
hmm something went wrong
hmm something went wrong
hmm something went wrong
hmm something went wrong
hmm something went wrong
hmm something went wrong


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [31]:
[p.name for p in fine_grained.particles()]

['C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'S',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'C'

In [ ]:
# Fresnel stuff

#scene = utils.draw_scene(cg_p3ht, show_atomistic=True, show_box=True, color="coolwarm")
#view = fresnel.interact.SceneView(scene)
#view.show()
#cam_pos = np.array([1,1,0])*np.array(p3ht.boundingbox.maxs)
#camera = fresnel.camera.Camera()
#camera.position = cam_pos
#camera.up = np.array([0,1,0])
#camera.height = max(p3ht.boundingbox.maxs)
#scene.camera = camera
#fresnel.pathtrace(scene, w=500, h=500, light_samples=10)